<a href="https://colab.research.google.com/github/danstudiohk/WebScraper/blob/main/Download_IG_Post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install beautifulsoup4

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from datetime import datetime

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 88.3 MB of archives.
After this operation, 294 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 99.0.4844.84-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 99.0.4844.84-0ubuntu0.18.04.1 [77.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 99.0.4844.84-0ubuntu0.18.0

In [3]:
ig = ['aiahongkongmacau',
      'fwd_hk',
      'prudential.hongkong',
      'chinalifeoversea',
      'bowtiehongkong',
      'axahongkong',
      'hsbc_hk',
      'citihongkong',
      'stancharthk',
      'beahk',
      'cncbi.bank'
      ]

In [7]:
def scrape_ig(ig):
    url = f'https://www.picuki.com/profile/{ig}'
    wd.get(url)
    LoopCnt = 20
    for j in range(1, LoopCnt):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    resp = wd.page_source
    soup = BeautifulSoup(resp, 'html.parser')
    content = []

    div_all = soup.find_all("div", {"class": "photo-description"})
    for d in div_all:
        content.append(d.text)

    date = []
    div_all = soup.find_all("div", {"class": "time"})
    for d in div_all:
        s_all = d.find_all('span')
        for s in s_all:
            date.append(s.text)

    like = []
    div_all = soup.find_all("div", {"class": "likes_photo"})
    for d in div_all:
        like.append(d.text.replace("\n",""))

    comment = []
    div_all = soup.find_all("div", {"class": "comments_photo"})
    for d in div_all:
        comment.append(d.text.replace("\n",""))

    df = pd.DataFrame(list(zip(date, content, like, comment)),
                columns =['ig_post_date', 'ig_content', 'ig_like_cnt', 'ig_comment_cnt'])
    df['ig_page'] = ig
    df = df[['ig_page','ig_post_date','ig_content', 'ig_like_cnt', 'ig_comment_cnt']]
    return df

df_aia = scrape_ig(ig[0])
df_fwd = scrape_ig(ig[1])
df_pru = scrape_ig(ig[2])
df_chinalife = scrape_ig(ig[3])
df_bowtie = scrape_ig(ig[4])
df_axa = scrape_ig(ig[5])
df_hsbc = scrape_ig(ig[6])
df_citi = scrape_ig(ig[7])
df_scb = scrape_ig(ig[8])
df_bea = scrape_ig(ig[9])
df_citic = scrape_ig(ig[10])

df_all = pd.concat([df_aia, df_fwd, df_pru, df_chinalife, df_bowtie, df_axa, df_hsbc, df_citi, df_scb, df_bea, df_citic])
df_all['ig_extract_date'] = datetime.now().strftime("%Y%m%d")


df_all.to_csv('/content/drive/MyDrive/HSBC Market Intelligence/ig'+ datetime.now().strftime("%Y%m%d%H%M%S") + '.csv')
df_all.to_csv('/content/drive/MyDrive/HSBC Market Intelligence/ig.csv')